In [3]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [4]:
! dvc pull

|data/prepared/test.tsv     672k/4.54M [00:01<00:32,     124kB/s]


 18%|█▊        |model.pkl                  464k/2.54M [00:01<00:42,    51.1kB/s]



 14%|█▍        |data/features/train.pkl    784k/5.41M [00:01<01:31,    53.2kB/s]




 53%|█████▎    |data/features/test.pkl     728k/1.35M [00:01<00:08,    77.8kB/s]





  6%|▌         |data/data.xml             2.00M/36.1M [00:01<02:59,     199kB/s]

  3%|▎         |data/prepared/train.tsv    504k/18.2M [00:01<03:11,    97.0kB/s]
 17%|█▋        |data/prepared/test.tsv     776k/4.54M [00:01<00:23,     168kB/s]


 22%|██▏       |model.pkl                  568k/2.54M [00:01<00:29,    71.4kB/s]

  4%|▍         |data/prepared/train.tsv    720k/18.2M [00:01<02:15,     136kB/s]




 86%|████████▌ |data/features/test.pkl    1.16M/1.35M [00:01<00:01,     110kB/s]



 17%|█▋        |data/features/train.pkl    944k/5.41M [00:01<01:03,    74.7kB/s]
 20%|█▉        |data/prepared/test.tsv     920k/4.54M [00:01<00:16,     225kB/s]





  6%|▌       

## Easily reproduce somewhere else

### from cache

In [5]:
! pipenv run dvc repro

Loading .env environment variables...
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Stage 'data/data.xml.dvc' didn't change, skipping
Stage 'prepare' didn't change, skipping
Stage 'featurize' didn't change, skipping
Stage 'train' didn't change, skipping
Stage 'evaluate' didn't change, skipping
Data and pipelines are up to date.


### or force actual pipeline run

In [6]:
! dvc repro -f

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Verifying data sources in stage: 'data/data.xml.dvc'

Running stage 'prepare' with command:
	python src/prepare.py data/data.xml
  0% Saving prepared|                          |0.00/2.00 [00:00<?,     ?file/s]
Running stage 'featurize' with command:
	python src/featurization.py data/prepared data/features
The input data frame data/prepared/train.tsv size is (20017, 3)
The output matrix data/features/train.pkl size is (20017, 502) and data type is float64
The input data frame data/prepared/test.tsv size is (4983, 3)
The output matrix data/features/test.pkl size is (4983, 502) and data type is float64
  0% Saving features|                          |0.00/2.00 [00:00<?,     ?file/s]
Running stage 'train' with command:
	python src/train.py data/features model.pkl
Input matrix size (20017, 502)
X matrix size (20017, 500)
Y matrix size (20017,)
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>
Running s

In [7]:
! dvc diff

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

### Assert equivalent result

In [8]:
! cat scores.json

{"auc": 0.5417487597055675}

### Now I can easily
* Making updates to the models
* see differences in parameters or results
* collaborate with others
* reproduce results

### Go and modify some parameters

In [9]:
! dvc status

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Data and pipelines are up to date.


In [10]:
! dvc params diff

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

In [11]:
! dvc repro

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Stage 'data/data.xml.dvc' didn't change, skipping
Stage 'prepare' didn't change, skipping
Stage 'featurize' didn't change, skipping
Stage 'train' didn't change, skipping
Stage 'evaluate' didn't change, skipping
Data and pipelines are up to date.


In [12]:
! dvc metrics diff

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

In [13]:
! cat scores.json

{"auc": 0.5417487597055675}

In [14]:
! dvc plots diff -x recall -y precision

file:///home/nichita/Projects/dvc_intro/plots.html


## For reproducibility of this notebook:
### Remove tracked data from dvc (local and remote) and
### Remove unnecessary files 

In [15]:
! dvc remove data/data.xml.dvc
! rm data/data.xml

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

In [16]:
! dvc diff

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Deleted:
    data/data.xml

files summary: 0 added, 1 deleted, 0 modified


In [17]:
! git add data/data.xml.dvc
! git commit -m 'removes data'
! git push origin tutorial
! dvc push

[data_pipelines 33548c6] removes data
 1 file changed, 3 deletions(-)
 delete mode 100644 data/data.xml.dvc
Everything up-to-date
Everything is up to date.


In [18]:
! echo "y" | dvc gc -cw

  0% Querying cache in 'gs://dvc_intro'|  |1.00/4.10k [00:00<11:11,  6.10file/s]

In [19]:
! echo 'y' | dvc destroy

This will destroy all information about your pipelines, all data files, as well as cache in .dvc/cache.
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

In [20]:
! rm -rf data/
! rm plots.html
! rm prc.json
! git add .
! git commit -m "clean"
! git push origin tutorial

[data_pipelines 98cbee6] clean
 10 files changed, 235 deletions(-)
 delete mode 100644 .dvc/.gitignore
 delete mode 100644 .dvc/config
 delete mode 100644 .dvc/plots/confusion.json
 delete mode 100644 .dvc/plots/default.json
 delete mode 100644 .dvc/plots/scatter.json
 delete mode 100644 .dvc/plots/smooth.json
 delete mode 100644 data/.gitignore
 delete mode 100644 dvc.lock
 delete mode 100644 dvc.yaml
 delete mode 100644 prc.json
Everything up-to-date
